In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/train.csv")

In [ ]:
df.head()

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
new_df = df.copy()

In [ ]:
gender_uniques = list(new_df["Gender"].unique())
new_df["Gender"] = new_df["Gender"].apply(lambda x: gender_uniques.index(x))

In [ ]:
corrMatrix = new_df.corr()
fig, ax = plt.subplots(figsize= (15,10))
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
new_df.head()

In [ ]:
vehicle_age_dict= {
    '> 2 Years' : 2,
    '1-2 Year' : 1,
    "< 1 Year" : 0
}
new_df["Vehicle_Age"] = new_df["Vehicle_Age"].apply(lambda x: vehicle_age_dict[x])

In [ ]:
new_df.head()

In [ ]:
dict_damage = {"Yes": 1, "No": 0}
new_df["Vehicle_Damage"] = new_df["Vehicle_Damage"].apply(lambda x: dict_damage[x])

In [ ]:
new_df.head()

In [ ]:
corrMatrix = new_df.corr()
fig, ax = plt.subplots(figsize= (15,10))
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
age =list(new_df["Age"])
responses = list(new_df["Response"])

In [ ]:
age_groups = list([(i-1, i) for i in range(1, 100, 1)])

In [ ]:
age_groups_response = {age_group: [0, 0] for age_group in age_groups}

In [ ]:
def get_age_group(age):
    for group in age_groups:
        if group[0] <= age <= group[1]:
            return group

In [ ]:
age_groups_response

In [ ]:
for a, r in zip(age, responses):
    group = get_age_group(a)
    age_groups_response[group][1] += 1
    if r == 1:
        age_groups_response[group][0] += 1

In [ ]:
age_groups_response

In [ ]:
groups, response_rate = zip(*list(map(lambda x: (x[0], x[1][0] / (x[1][1]+1)),age_groups_response.items())))

In [ ]:
groups

In [ ]:
response_rate

In [ ]:
#groups_string = list([f"{group[0]}-{group[1]}" for group in groups])
groups_string = []
for i in range(len(groups)):
    group = groups[i]
    if i % 5 ==0:
        groups_string.append(f"{group[0]}")
    else:
        groups_string.append("")

# Age groups have correlation the positive response is much better at ages 30 - 50

In [ ]:
plt.bar(list([i for i in range(len(response_rate))]), response_rate, tick_label=groups_string)

In [ ]:
mean_response_rate = responses.count(1) / len(responses)

In [ ]:
mean_response_rate

In [ ]:
plt.bar(list([i for i in range(len(response_rate))]), response_rate, tick_label=groups_string, label="response_rate")
plt.plot([0,100], [mean_response_rate, mean_response_rate],c="grey", label="mean response rate")
plt.xlabel("age")
plt.ylabel("response rate")
plt.legend()

In [ ]:
age_groups = list([(0, 30), (30, 50), (50 , 60), (60, 100)])

In [ ]:
def get_age_group(age):
    for group in age_groups:
        if group[0] <= age <= group[1]:
            return group

In [ ]:
ages = list(new_df["Age"])

In [ ]:
age_groups_l = [[] for _ in range(len(age_groups))]
for age in ages:
    group = get_age_group(age)
    i = age_groups.index(group)
    age_groups_l[i].append(1)
    all_indices = list([i for i in range(len(age_groups_l))])
    all_indices.remove(i)
    for index in all_indices:
        age_groups_l[index].append(0)
    

In [ ]:
age_groups_l

In [ ]:
for group, group_name in zip(age_groups_l, age_groups):
    group_str = f"age group {group_name[0]} -{group_name[1]}"
    new_df[group_str] = group

In [ ]:
new_df.head(10)

In [ ]:
corrMatrix = new_df.corr()
fig, ax = plt.subplots(figsize= (15,10))
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
rc = list(new_df["Region_Code"])
response = list(new_df["Response"])

In [ ]:
new_df["Region_Code"].describe()

In [ ]:
region_codes = list(set(rc))

In [ ]:
region_codes_dict = {r: [0, 0]for r in region_codes}

In [ ]:
def categorical_data_to_groups(data, responses):
    uniques = list(set(data))
    data_response_dict = {d: [0, 0] for d in uniques}
    for region_code_, r in zip(data, responses):
        if r==1:
            data_response_dict[region_code_][0] += 1
        data_response_dict[region_code_][1] += 1
    region_codes, response_rates = zip(*list(map(lambda x: (x[0], x[1][0] / (x[1][1]+0.1)) , list(data_response_dict.items()))))
    return region_codes, response_rates

In [ ]:
for region_code_, r in zip(rc, response):
    if r==1:
        region_codes_dict[region_code_][0] += 1
    region_codes_dict[region_code_][1] += 1

In [ ]:
region_codes_dict

In [ ]:
region_codes, response_rates = zip(*list(map(lambda x: (x[0], x[1][0] / (x[1][1]+0.1)) , list(region_codes_dict.items()))))

In [ ]:
plt.bar(list([i for i in range(len(response_rates))]), response_rates)
plt.plot([0,len(response_rates)], [mean_response_rate, mean_response_rate],c="grey", label="mean response rate")

In [ ]:
sorted_response_rate = sorted(list(zip(region_codes, response_rates)), key= lambda x: x[1], reverse=True)

In [ ]:
region_codes_groups = ["top region","normal region", "bad region"]
sizes = [(0,10), (10, 30), (30, 53)]

In [ ]:
region_codes_sorted, response_rates = zip(*sorted_response_rate)

In [ ]:
top_group = region_codes_sorted[:sizes[0][1]]
normal_group = region_codes_sorted[sizes[1][0]:sizes[1][1]]
bad_group = region_codes_sorted[sizes[2][0]:]
# region_codes_g = [top_group, normal_group, bad_group]

In [ ]:
bad_group

In [ ]:
groups_ = [top_group, normal_group, bad_group]

In [ ]:
arrays = [[], [], []]
for region_code_, r in zip(rc, response):
    for i in range(len(groups_)):
        arrays[i].append(int(region_code_ in groups_[i]))

In [ ]:
for counter, ar in enumerate(arrays):
    name = region_codes_groups[counter]
    new_df[name] = ar

In [ ]:
new_df.head()

In [ ]:
plt.boxplot(response_rates)

In [ ]:
corrMatrix = new_df[["Response", "top region", "normal region", "bad region"]].corr()
fig, ax = plt.subplots(figsize= (15,10))
sn.heatmap(corrMatrix, annot=True)
plt.show()

Top!! Region Code works!

In [ ]:
new_df.head()

In [ ]:
data = list(new_df["Policy_Sales_Channel"])
responses = list(new_df["Response"])

In [ ]:
data_values, response_rates = categorical_data_to_groups(data, responses)

In [ ]:
plt.title("Policy Sales Channel Response Rates")
plt.bar(list([i for i in range(len(response_rates))]), response_rates)
plt.plot([0,len(response_rates)], [mean_response_rate, mean_response_rate],c="grey", label="mean response rate")
plt.xlabel("Policy Sales Channels")
plt.ylabel("response rates")

